<a href="https://colab.research.google.com/github/sunkusun9/ML2022_1/blob/main/TPS202208_Prob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

[Kaggle의 Tabular Playground 2022-08월 과제](https://www.kaggle.com/competitions/tabular-playground-series-aug-2022) 를 기반으로 만들어진 문제입니다.

액체가 엎질러졌을 제품에 따라서 여러 조건에 따라, 물기를 완전히 흡수하는데 성공할지 아니면 실패했는지에 대한 테스트하는 실험에 대한 데이터입니다. 


|속성명|설명|타입|
|------|----|----|
|ID|고유번호|int|
|loading|흡수할 용량|float|
|product_code|상품코드|string|
|attribute_0|구성물질 1|string|
|attribute_1|구성물질 2|string|
|attribute_2|구성물질 1 함량|string|
|attribute_3|구성물질 2 함량|string|
|measure_0~2|측정치 0~2|int|
|measure_3~17|측정치 3~17|float|
|failure|실패여부|boolean| 

# Download

Kaggle API를 이용하여 Kaggle에 등록된 ID와 인증키를 가지고 데이터를 다운로드

In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''

# train.csv가 없으면 Kaggle에서 데이터를 다운로드 받는다.
if not os.path.exists('train.csv'):
    !kaggle competitions download -c tabular-playground-series-aug-2022
    !unzip tabular-playground-series-aug-2022.zip
    !rm tabular-playground-series-aug-2022.zip

  0% 0.00/2.27M [00:00<?, ?B/s]
100% 2.27M/2.27M [00:00<00:00, 164MB/s]
Archive:  tabular-playground-series-aug-2022.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!head train.csv

id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,A,80.1,material_7,material_8,9,5,7,8,4,18.04,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,,13.034,14.684,764.1,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.54,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.24,12.718,18.288,12.715,15.607,,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.06,12.471,16.346,18.377,10.02,15.25,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.95,16.99,15.746,11.306,1

# Preparation

본 문제는 

|패키지|버젼|
|------|----|
|pandas|1.3.5|
|numpy|1.21.6|
|sklearn|1.0.2|
|scipy|1.7.3|

기반으로 만들어 졌습니다. 문제에서 Hint로 제공하는 수치는 위 버젼과 다르면 다를 수 있습니다.

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import scipy

for i in [pd, np, sklearn, scipy]:
    print(i.__name__, i.__version__)

pandas 1.3.5
numpy 1.21.6
sklearn 1.0.2
scipy 1.7.3


# 전처리

measurement_17는 failure를 유추하기 위해 product_code 별로 measurement3 ~ measurement_9을 최적의 선형적인 조합으로 기기에서 만들어 주는 값이다. 즉 product_code 별로 measurement3 ~ mesuremenet_9을 입력으로 measurement_17을 선형회귀모델로 학습하면 measurement_17을 복원하는 모델이 된다. 하지만 실험 기기가 종종 오류를 일으켜 measurement_17에 결측이 발생한다. 이 정보를 바탕으로 복원을 위한 선형회귀모델을 만들어 measurement_17의 결측치를 보간하고자 한다.

※ 본 처리 과정은  [Perfect Positive Correlation with measurement_17](https://www.kaggle.com/competitions/tabular-playground-series-aug-2022/discussion/343939) 을 기반으로 만들어 졌습니다.

## 단계 1

measurement_3~9 중에서 결측치가 전혀없는 examine 데이터프레임을 만든다.

## 단계 2

examine 데이터프레임을 활용하여, product_code 별로 measurement_3 ~ measurement9  속성을 입력으로 하여 measurement_17을 맞추는 선형회귀모델을 만들고, 이 모델들을 활용하여 복원할 수 있는 measurement_17 결측치를 최대한 복원한다. 그리고 이 데이터프레임을 basetable이라고 한다.

- 선형 모델읜 sklearn.linear_regression.Lasso를 사용하고, 모든 설정은 기본값을 사용한다.

- 회귀계수의 값이 0인 속성은 measurement_17과 전혀 관련이 없다고 간주한다. 

- 절편을 제외한 각 속성에 해당하는 회귀계수의 총합은 1203.35(소수점 2째 자리까지 반올림)이다.

 Guide. 속성명을 열의 이름으로, product_code를 행의 Index로 하여 각각의 값이 회귀계수인 데이터프레임 coef를 만들고 여기서 합을 내도록함을 권장.
 > 단계 3에서도 활용할수 도 있음

* Hint
    - 복원 후 measurement_17의 결측치는 341개로 줄어 든다.

## 단계 3 (Optional)

※ 본 단계는 구현의 난이도가 상당하여 Optional 단계로 하게 되었고 더 깊은 연습을 하시고자 하는 분들을 위해 만든 것이니, 단계에 영향이 없도록, basetable에서 optional 데이터프레임을 만들고 진행토록합니다. 


위에서 발견한 거의 완벽한 선형적 관계를 이용하여 measurement_17 이외에도 복원할 수 있는 값은 모두 복원하고자 한다. measurement_3~measurement_9 까지 복원하시오.

- sklearn.linear_model.LinearRegression을 사용한다.

- optional 데이터프레임에서 결측을 제거하여 examine 데이터프레임을 새로 만들고 이를 활용하여 선형모델을 만든다.

- 결측 처리를 완료하면, measurement_3 ~ measurement_9 까지의 결측치는 모두 2294건이 된다.

# 문제 1.

(basetable을 사용) measurement_3~16 까지 결측 여부가 failure에 영향이 있는 지를 파악하고자 한다.

이를 확인하기 위하여, 

n이 3 부터 16까지, 다음의 검증을 수행한다.

$H_0: P(measurement\_\{n\}=Missing|failure=True) = P(measurement\_\{n\}=Missing|failure=False)$

$H_1: P(measurement\_\{n\}=Missing|failure=True) \not= P(measurement\_\{n\}=Missing|failure=False)$

※ 본 문제는 kaggle의 [TPSAUG22 EDA which makes sense](https://www.kaggle.com/code/ambrosm/tpsaug22-eda-which-makes-sense) 코드 분석을 기반으로 만들어졌습니다.


## 단계 1.

우선, measurement_3 속성 하나로 위 검증을 시행하자.

P_missing_true는 failure가 1일 때 measurement_3가 결측일 확률

P_missing_false는 failure가 0 일때 measurement_3가 결측일 확률


이라고 할 때 이 값을 구하라.

## 단계 2

위 가설은 다음과 같이 

$H_0: P\_missing\_false - P\_missing\_true = 0$

$H_1: P\_missing\_false - P\_missing\_true \not= 0$

으로 바꿀 수 있다. 

중심극한정리(Central Limit Theorem)이 충분히 성립함이 이견이 없다고 할 때, 

P_missing_false - P_missing_true의 분산을 구하고 이를 바탕으로 구하고, 

정규분포를 사용하여 위 가설에 대한 p-value를 구하라.

* Hint: scipy.stats.norm 모듈을 활용

## 단계 3

measuremenet_4 ~ 9 까지 위 검증을 반복하고 결측치가 유의한 차이를 보인 속성의 p_value의 합을 구하여 A라고 한다. (유의 수준은 5%로 한다.)

결측치에 대한 유의한 차이를 보이는 속성은 총 두 개이고, 속성 번호가 작은 것에 대한 결측유무의 속성명을 na_1으로 큰 것을 na_2로 하는 속성 2개을 (basetable)에 추가하고 (결측: 1, 비결측: 0), 이 데이터셋의 명을 prob_1 이라고 한다

A의 값을 소수점 3째 짜리까지 반올림하여 출력한다.

Ex) 0.052

## 단계 4 (Optional)

measurement_3~measurement_16까지, 결측치수가 failure와 연관성이 있을지 검증하시오.

# 문제 2



두가지 사항을  확인하고자 한다.

첫째는 실험에 대해 공평하게 진행이 됐는지를 의문을 가지고 있어 product_code에 대해서 loading을 동일한 조건으로 진행했는지를 조사하라.

분산분석(ANOVA)을 통해 진행을 해보고, 조사의 적합성을 살펴보기 위해, 분산분석의 가정에 얼마나 부합하는지를 보고하라고 덧붙였다. 

둘째는, 조사결과 attribute_0와 attribute_1은 failure와는 관계가 있는지를 조사하는데,해당 재료들의 조합은 이전의 여러 실험에서 성능에 차이가 없다는 결과가 있어 유의 수준을 1%로 정했다.


## 단계 1

prob_1에서 loading 값이 결측이 아닌 값을 뽑아 prob_2 데이터프레임을 만들고, 검증은 prob_2를 통해 진행한다.

prob_2에 loading 속성에 로그 함수를 적용하여 loading_log 컬럼을 만들어, 좌측으로 치우친 loading의 분포를 보다 정규분포 형태로 변환을 한다.

- np.log 함수를 사용한다.

## 단계 2

(prob_1을 이용) 일원분산분석을 진행을 위해서, loading_log 열이 일원산분산분석의 가정인  product_code 내에서 동일한 분산이 성립하는지 조사하고자 한다.

Bartlett 검정 방법으로, 조사했을 때의  pvalue를 구하고 이 값을 B라고 한다.

- scipy.stats.bartlett 함수를 이용라.

## 단계 3

loading_log가 product_code 각각에 대해 정규성을 지니고 있는지 확인하고자 한다. jarque_bera test로 검증하고 결과의 p-value가 0.05가 넘는 열의 수를 C라고 하자.

- scipy.stats.jarque_bera 함수를 이용하라.

## 단계 4

product_code에 대해서 분산분석을 통해서 loading_log 평균의 차이가 나는 지를 확인하고, 해당 검증의 pvalue를 D라고 한다.

- scipy.stats.f_oneway 를 이용하라.

## 단계 5

chi2 분포를 이용한 동질성 검사를 통해 attirubute_0, attribute_1의 조합이 failure에 영향이  있는지 chi2 동질석 검사를 진행하라. 연관성이 있다고 할 수 있으면 E 값은 1로 아니면 0으로 한다.

- 유의 수준은 1%로 한다.

- scipy.stats.chi2_contingency 를 이용하고, correction은 False로 한다.

B + C + D + E 의 값을 소수점 3째 자리에서 반올림하여 2째 자리까지 출력하라. 합을 구한 후에 반올림을 진행해야 한다는 점을 유의하시오.

예) 3.16

# 문제 3

로지스틱회귀모델로 수치형 measurement_0 ~ measurement_17과 loading와 불린형인 na_1, na_2 입력으로 하고, failure를 타깃으로 하는 분류 모델을 제작을 했는데, 제 성능을 내지 못했다.  다른 모델을 사용해보기 전에 효과가 있을 것이라고 생각되는 속성인 loading과 measurement_17으로 시작하여 효과가 있는 조합을 찾고자 한다. 선택되지 않은 속성을 하나 씩만 넣어보고 가장 효과가 큰 속성이 이전 단계보다 좋은 성능 냈다면 해당 속성을 추가 하고 선택 과정을 반복하고 그렇지 않으면 선택을 중단하는 전진선택법을 취하여 속성 조합을 찾기 위해 다음 단계를 진행하라.

대상지표는 AUC(Area Under the ROC Curve) 를 사용한다.

- sklearn.metrics.roc_auc_score

## 단계 1

prob_1 에서 80%는 학습 데이터 prob_3_train과 20%는 테스트 데이터 prob_3_test로  나누고, 표준스케일링 방법으로 학습 데이터의 입력 속성들 중에서 **수치형** 속성을 정규화하고, 학습 데이터에서 사용했던 표준스케일링 설정으로 테스트 데이터의 수치형 입력 속성들을 정규화한다. 

- sklearn.model_selection.train_test_split 함수 이용, random_state=123로 설정한다.

- sklearn.preprocessing.StandardScaler

## 단계 2



결측치 처리 방법를 모두 0 (표준 정규화를 했으므로 평균값이다) 으로 대체한 것과, KNNImputer를 사용한 것과의 성능차이를 비교하여 성능이 좋은 쪽으로 결측치를 처리한 다음에 진행하고자 한다. 모든 속성을 넣었을 때 로지스틱회귀모델은 제대로 된 결과를 내놓지 못하여, 이 보다 유연하게 구분 영역을 만들 수 있는 Decision Tree Classifier를 사용한다. 입력은 measurement_0 ~ measuremenet_17, loading, na_1, na_2 순으로 한다. 
최종적으로 좋은 성능을 보인 결측치 처리 방법으로 prob_3_train, prob_3_test를 교정한다. 문제 4에서도 이 데이터 셋을 사용하여 진행할 예정이다.

- sklearn.tree.DecisionTreeClassifier random_state=123, max_depth=5 로 한다.

- sklearn.impute.KNNImputer

- df_prob_3_train, df_prob_3_test는 성능 비교 후에 처리 될 수 있도록 유의하시오

## 단계 3



로지스틱회귀모델을 사용하여 입력 변수 중에서, loading과 measurement_17 만을 사용하여 학습하고 테스트에 대한 성능을 측정하라. 하고 해당 값을 best_auc 변
수에 저장한다.


- sklearn.linear_model.LogisticRegression 모든 설정은 기본


## 단계 4

문제 3 서두에서 설명한 속성 선택법으로 속성을 선택하고 best_auc를 갱신하고 이 값을 소수점 넷째 자리에서 반올림하여, 3째 짜리까지 출력하라. 그리고, 테스트 결과에 대한 failure가 1일 확률을 저장하여 prob_3_test에 저장한다.

Ex) 0.123

## 단계 5 (Optional)

전처리 단계 3에서 나온 df_optional 데이터프레임으로 KNNImputer를 사용하여 단계 1과 2에서 KNNImputer로 했을 경우와 성능을 비교해보자. 좋은 성능이 나왔다면
속성 선택 과정까지 진행해보자.

# 문제 4



PCA를 사용하여 차원 감소로 어떤 효과를 볼 수 있을 지 검증하고자 한다.

## 단계 1

진행하기 전에 loading을 포함하지 포함하지 않을지를 확인하고자 한다. 분석가들 사이에 loading은 흡수력하고는 상관이 없으니 빼고 해야 하는 쪽과 loading에 따라 흡수력 세팅을 달리 했을 테니 포함하는 것이 좋다는 쪽으로 나뉘 었다. 이를 위해 흡수력에 따른 loading과의 상관도를 파악해 보기 위해 LinearDiscriminantAnalysis를 통해 loading을 제외하고 failure를 나누었을 때 나온 투영점과 loading과의 spearman correlation을 구하고, 이 때 pvalue를 F에 저장한다. prob_3_train 데이터프레임을 사용한다.

- sklearn.discriminant_analysis.LinearDiscriminatAnalysis
- scipy.stats.spearmanr

## 단계 2

prob_3_train으로 measurement_0~measurement_17까지 PCA를 진행하고, 50%이상의 분산을 설명할 수 있는 pca의 컴포넌트의 수를 G에 저장한다.

- sklearn.decomposition.PCA

## 단계 3

로지스틱회귀 분석을 사용하여 'loading' 속성과 PCA 컴포넌트수를 G부터 증가시켜서 failure에 대한 roc_auc_score에 대한 가장 좋은 prob_4_test에 대한 성능을 보이는 컴포넌트수 H를 구하라

F + G + H 를 소수점 셋째 자리에서 반올림하여, 2째 자리까지 구하라.

# 문제 5

[Kaggle 형] train.csv을 학습 데이터로 사용하여, failure를 맞추는 분류 모델을 만들고, test.csv 테스트 데이터에 대한 failure일 확률을 다음과 같은 포맷으로 submission.csv라는 이름으로 만들어라. 측정에 대한 지표는 area under of ROC curve 이다.

Id,failure

26570, 0.1

26571, 0.2

....




# 문제 6 [Optional]


학습 / 검증 분리를 통한 결과 보다 치우침이 없는 교차 검증을 사용하여 최적화된 모델을 탐색하고자 한다. 다음과 같은 과정으로 진행한다.

## 단계 1
prob_1 데이터셋을 prob_6 데이터셋으로 복사하고, 문제 3의 2단계에서 수행했던, KNNImputer를 통한 결측치 보간을 진행한다.

- sklearn.imputer.KNNImputer

## 단계 2

loading과, measurement_0~measurement_17은 표준정규화를 수행하고, 학습에  대한 AUC와  테스트에 대한 failure가 1일 확률을 예측한 결과를 넘겨주는 함수를 만들어라.

- sklearn.preprocessing.StandardScaler
- sklearn.metrics.roc_auc_score

## 단계 3

product_code에 대한 그룹 5-fold 교차검증을 2단계에서 작성한 함수를 사용하여 LogisticRegression으로 진행한다.

- sklearn.model_selection.GroupKFold, random_state=123
- loading, measurement_0~measurement_17, na_1, na_2를 입력으로 하라.

## 단계 4

최적의 RandomForestClassifier의 Hyper Parameter Tuning을 진행하고자 한다. 

탐색할 매개 변수는, 

n_estimators=[5, 10, 15]와, Decision Tree의 max_depth=[5, 6, 7], 그리고 min_samples_split = [512, 1024] 로 한다.

* sklearn.ensemble.RandomForesetClassifier, random_state=123
* itertools의 product를 이용
* loading, measurement_0~measurement_17, na_1, na_2를 입력으로 순서에 맞춰 진행 하라.

교차 검증을 통해 도출된 최적의 auc에 해당하는 
n_estimators + max_dpeth + min_samples_split + 최적 AUC를 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하라.